# Что тут происходит? Зачем вообще нужен этот файл?

Скорость обработки видео с помощью `DAMO-NLP-SG/VideoLLaMA3-7B` составляет примерно 100 видео в час. Чтобы распараллелить этот процесс мы параллельно запускали несколько ноутбуков (внутри одного блокнота максимум 3 параллельных сессии). Этот файл нужен для того, чтобы объединить те файлы .json, которые мы получили для различных групп видео. Также, здесь для удобства реализована сортировка файлов в том же порядке, в котором они представлены в [таблице](https://s3.ritm.media/hackaton-itmo/task.csv).

Формат JSON'а на вход (тот файл, который был получен после запуска `vide_to_text.ipynb`):

```
[
  {
    "id":"1",
    "filename": "bde2e7b0-0774-4a72-bb3b-57515512acd9.mp4",
    "description": "Тут хранится текстовое описание по видео.",
    "transcription": "Тут хранится текстовое описание по аудио."
  },
  ...
]
```

Формат выходного JSON соответствует тому, что указан выше, тут происходит просто объединение файлов.

Пример структуры файлов:

```
./files_to_merge
    |-- video_descriptions_0_200.json
    |-- video_descriptions_200_2390.json
    |-- video_descriptions_2400_2600.json
    |-- video_descriptions_2600_2800.json
    |-- video_descriptions_2800_3000.json
    |-- video_descriptions_3000_3200.json
    |-- video_descriptions_3200_3400.json
    |-- video_descriptions_3400_3600.json
    `-- video_descriptions_3600_3800.json
```

**ВАЖНО** : Перед запуском файла необходимо загрузить в папку `files_to_merge` те файлы, которые надо объединить.

# Запуск

In [8]:
import os
import json
import pandas as pd

# Папка с JSON-файлами
merge_dir = "files_to_merge"

# Загрузка таблицы
csv_url = "https://s3.ritm.media/hackaton-itmo/task.csv"
df_order = pd.read_csv(csv_url)

# Извлекаем имя файла из URL
df_order["filename"] = df_order["video_link"].apply(lambda x: x.strip().split("/")[-1])

# Загружаем все JSON-файлы
all_data = []
for file in os.listdir(merge_dir):
    if file.endswith(".json"):
        with open(os.path.join(merge_dir, file), "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
                all_data.extend(data)
            except Exception as e:
                pass  # можно включить логирование при необходимости

# Преобразуем к словарю по filename
data_by_filename = {item["filename"]: item for item in all_data if "filename" in item}

# Финальный список
merged_ordered = []
for idx, filename in enumerate(df_order["filename"], start=1):
    item = data_by_filename.get(filename)
    if item:
        result_item = {
            "id": idx,
            "filename": filename,
            "description": item.get("description", ""),
            "transcription": item.get("transcription", "")
        }
        merged_ordered.append(result_item)

# Сохраняем файл
output_filename = f"merged_ordered_{len(merged_ordered)}.json"
with open(output_filename, "w", encoding="utf-8") as f:
    json.dump(merged_ordered, f, ensure_ascii=False, indent=2)

print(f"Итог сохранён в: {output_filename}")

Итог сохранён в: merged_ordered_10040.json


# Скачать JSON

In [ ]:
from google.colab import files
files.download(output_filename)